In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
credentials_path = './../pamf-dwh-775beaae50ff.json'
project_id = 'pamf-dwh'

sql = """SELECT * FROM `pamf-dwh.DFS_VIEW.deboursement`"""
savings = pd.read_gbq(sql, project_id = project_id, private_key = credentials_path, dialect = 'standard')

deb = savings.copy()

renouvel = pd.read_csv('./renouvellement(august).csv')

temp=deb.merge(renouvel , left_on='rContactID' , right_on='rContactIDc')

temp[(temp.Status==4) & (temp.disbursement_date <='2019-08-20')]

temp

In [2]:
sql = """SELECT * FROM `pamf-dwh.DFS_VIEW.saving`"""

savings = pd.read_gbq(sql, project_id = project_id, private_key = credentials_path, dialect = 'standard')

E:\Anaconda3\lib\site-packages\pandas\io\gbq.py:149: FutureWarning: private_key is deprecated and will be removed in a future version.Use the credentials argument instead. See https://pandas-gbq.readthedocs.io/en/latest/howto/authentication.html for examples on using the credentials argument with service account keys.
  credentials=credentials, verbose=verbose, private_key=private_key)


In [3]:
#savings.Transaction_Date = savings.Transaction_Date.dt.tz_localize(None)
savings['Transaction_Date'] = savings.Transaction_Date.dt.tz_localize(None)
savings['Transaction_Time'] = savings.Transaction_Time.dt.tz_localize(None)

def combine_time(transaction_date, transaction_time):
    if pd.isnull(transaction_date) == pd.isnull(pd.NaT):
        return 0
    else:
        return pd.datetime.combine(transaction_date,transaction_time.time())

savings['transaction_date'] = savings.apply(lambda r : combine_time(r['Transaction_Date'], r['Transaction_Time']), axis = 1)

savings.drop(columns=['Transaction_Date','Transaction_Time'],inplace=True)
savings.sort_values('transaction_date', inplace=True)

In [4]:
savings = savings.query('Transaction_ID != "ManualCorrection_ReconIssueLoansSecoundPaymentZero"')

In [5]:
#savings[savings['Transaction_ID']=='ManualCorrection_ReconIssueLoansSecoundPaymentZero']

In [6]:
#savings.drop(savings.query('Transaction_ID == "ManualCorrection_ReconIssueLoansSecoundPaymentZero"'), inplace=True)

### Pre-balance and Post-balance

In [7]:
savings['post_balance']=savings.groupby('rContactID').cumsum()['Total_Payment'] #agg({'Total_Payment':'cumsum'})
savings['prev_balance']=savings['post_balance'] - savings['Total_Payment']

# - Putting the cycle

In [8]:
savings['is_less_200Ar_and_withdrawed']=(savings.post_balance<200) & (savings.Total_Payment<0)

savings['is_less_200Ar_and_withdrawed'] = savings['is_less_200Ar_and_withdrawed'].map( {True: 1, False: 0})



cycle = savings.groupby('rContactID').agg({'is_less_200Ar_and_withdrawed': 'sum'})
cycle = cycle.reset_index()

cycle.rename(columns={'is_less_200Ar_and_withdrawed':'cycle'}, inplace = True)

savings = savings.merge(cycle, how='inner', on=['rContactID'])

savings['nth_cycle'] = (savings.groupby('rContactID')['is_less_200Ar_and_withdrawed'].cumsum()) # -1 #.agg({'is_less_200Ar_and_withdrawed':'cumsum'})

# - Getting the max post bal per cycle

In [9]:
mx_pst_bal = savings.groupby(['rContactID','nth_cycle'],as_index=False).agg({
        'post_balance':'max'
}).groupby(['rContactID']).mean()

In [10]:
mx_pst_bal.reset_index(inplace=True)

In [11]:
mx_pst_bal.describe()

,rContactID,nth_cycle,post_balance
count,18067.000000,18067.000000,1.806700e+04
mean,73945.374440,0.971799,4.549693e+04
std,49472.835663,1.530266,2.174172e+05
min,23.000000,0.000000,5.000000e-01
25%,29722.000000,0.500000,8.000000e+02
50%,70071.000000,0.500000,5.000000e+03
75%,113910.000000,1.000000,2.675083e+04
max,171597.000000,49.500000,9.295200e+06


In [12]:
mx_pst_bal[mx_pst_bal['post_balance']<0]

,rContactID,nth_cycle,post_balance


In [13]:
savings[savings['rContactID']==1048]# ['Transaction_ID'] #[45488] #.sum()

,rContactID,Account_Number,Balance,Transaction_Type,Total_Payment,Transaction_ID,Status,saving_cycle,transaction_date,post_balance,prev_balance,is_less_200Ar_and_withdrawed,cycle,nth_cycle
84170,1048,00000000940,0.0,WalletToAccount,1500.0,153761046809014187,Success,1,2018-09-22 13:01:13.736,1500.0,0.0,0,4,0
84171,1048,00000000940,0.0,WalletToAccount,1000.0,153762255799081182,Success,2,2018-09-22 16:22:43.610,2500.0,1500.0,0,4,0
84172,1048,00000000940,0.0,AccountToWallet,-500.0,153763625136761933,Success,3,2018-09-22 20:10:57.050,2000.0,2500.0,0,4,0
84173,1048,00000000940,0.0,AccountToWallet,-500.0,153763833914278402,Success,4,2018-09-22 20:45:44.796,1500.0,2000.0,0,4,0
84174,1048,00000000940,0.0,AccountToWallet,-500.0,153789041732407955,Success,5,2018-09-25 18:47:04.100,1000.0,1500.0,0,4,0
84175,1048,00000000940,0.0,AccountToWallet,-200.0,153821534731913807,Success,6,2018-09-29 13:02:33.950,800.0,1000.0,0,4,0
84176,1048,00000000940,0.0,AccountToWallet,-500.0,153831827592454775,Success,7,2018-09-30 17:38:02.733,300.0,800.0,0,4,0
84177,1048,00000000940,0.0,AccountToWallet,-100.0,153945187742449815,Success,8,2018-10-13 20:31:26.120,200.0,300.0,0,4,0
84178,1048,00000000940,0.0,AccountToWallet,-200.0,154020822425879167,Success,9,2018-10-22 14:37:14.223,0.0,200.0,1,4,1
84179,1048,00000000940,0.0,WalletToAccount,120000.0,154520627421250071,Success,10,2018-12-19 10:58:37.806,120000.0,0.0,0,4,1


In [14]:
mx_pst_bal.rename(columns={'post_balance':'mean_post_balance_per_cycle'},inplace=True)

In [15]:
savings = savings.merge(mx_pst_bal[['rContactID','mean_post_balance_per_cycle']], on='rContactID', how = 'inner')

In [16]:
savings[savings[['rContactID','saving_duration','cycle','mean_post_balance_per_cycle']]['mean_post_balance_per_cycle']>0].sample(10)

KeyError: "['saving_duration'] not in index"

In [17]:
savings[savings['rContactID']==79338].Total_Payment.sum()

0.0

In [18]:
len(savings)

201788

In [71]:

dd = savings.groupby('rContactID').agg(
    {"transaction_date":"max"
        
    }
    )
dd.rename({'transaction_date':'last_payment'}, axis=1, inplace=True)

dd = dd.reset_index()


savings = savings.merge(dd, how = 'inner', on='rContactID')

In [72]:
sav = savings.groupby('rContactID').agg(
    {'Total_Payment':'sum','transaction_date':('min', 'max'), 'cycle':'max'})
sav.columns = ["_".join(x) for x in sav.columns.ravel()]
#dd.columns = dd.columns.droplevel(0)
sav = sav.reset_index()
sav.rename(columns={'Total_Payment_sum':'balance'}, inplace = True)

sav['saving_duration'] = sav['transaction_date_max']-sav['transaction_date_min'] 
#if pd.isnull(saving_duration) != pd.isnull(pd.NaT):
sav.saving_duration = sav.saving_duration.apply(lambda x: str(x))

sav.saving_duration = sav.saving_duration.apply(lambda x: str.split(x)[0])

savings.transaction_date = savings.transaction_date.apply(lambda x: pd.to_datetime(x))
savings.last_payment = savings.last_payment.apply(lambda x: pd.to_datetime(x))

# RFM

In [19]:
"""
import datetime as dt
NOW = dt.datetime(2011,12,10)df1['InvoiceDate'] = pd.to_datetime(df1['InvoiceDate'])

rfmTable = df1.groupby('CustomerID').agg({'InvoiceDate': lambda x: (NOW - x.max()).days, 'InvoiceNo': lambda x: len(x), 'TotalPrice': lambda x: x.sum()})rfmTable['InvoiceDate'] = rfmTable['InvoiceDate'].astype(int)
rfmTable.rename(columns={'InvoiceDate': 'recency', 
                         'InvoiceNo': 'frequency', 
                         'TotalPrice': 'monetary_value'}, inplace=True)

"""

"\nimport datetime as dt\nNOW = dt.datetime(2011,12,10)df1['InvoiceDate'] = pd.to_datetime(df1['InvoiceDate'])\n\nrfmTable = df1.groupby('CustomerID').agg({'InvoiceDate': lambda x: (NOW - x.max()).days, 'InvoiceNo': lambda x: len(x), 'TotalPrice': lambda x: x.sum()})rfmTable['InvoiceDate'] = rfmTable['InvoiceDate'].astype(int)\nrfmTable.rename(columns={'InvoiceDate': 'recency', \n                         'InvoiceNo': 'frequency', \n                         'TotalPrice': 'monetary_value'}, inplace=True)\n\n"

In [22]:
from datetime import date
today = date.today()
print("Today's date:", today)

today = pd.to_datetime(today)
today = today.tz_localize(None)
first_submission = pd.datetime(2018,3,26)

max_duration_allowed = abs(first_submission - today)

Today's date: 2019-08-26


In [23]:
rfmTable = savings.groupby('rContactID').agg({'transaction_date': lambda x: abs((x.max()-today).days), 
                                           'cycle': 'max'}) 
                                           #'mean_post_balance_per_cycle': lambda x: x.mean()})

In [24]:
rfmTable = rfmTable.merge(mx_pst_bal[['rContactID','mean_post_balance_per_cycle']], on ='rContactID', how = 'inner')

In [26]:
rfmTable

,rContactID,transaction_date,cycle,mean_post_balance_per_cycle
0,23,388,1,2.500000e+02
1,24,481,3,6.522625e+02
2,25,13,14,5.245400e+04
3,26,64,1,1.504500e+06
4,27,4,2,2.279300e+04
5,28,374,2,1.100000e+04
6,29,282,3,5.870000e+03
7,30,199,6,8.787086e+04
8,31,505,0,1.000000e+05
9,32,442,4,4.200480e+04


In [27]:
rfmTable['transaction_date'] = rfmTable['transaction_date'].astype(int)
rfmTable.rename(columns={'transaction_date':'recency',
                         'cycle':'frequency',
                         'mean_post_balance_per_cycle':'monetary_value'}, inplace=True)

In [28]:
#rfmTable.monetary.quantile([0.25, 0.5, 0.75])

## - should use the monetary value quantile as condition

# - Rectification:

In [29]:
quantiles = rfmTable.quantile(q=[0.25,0.5,0.75])
quantiles = quantiles.to_dict()

In [30]:
quantiles

{'rContactID': {0.25: 29722.0, 0.5: 70071.0, 0.75: 113910.0},
 'recency': {0.25: 28.0, 0.5: 111.0, 0.75: 245.0},
 'frequency': {0.25: 1.0, 0.5: 1.0, 0.75: 2.0},
 'monetary_value': {0.25: 800.0, 0.5: 5000.0, 0.75: 26750.833333333336}}

# - Create a segmented RFM table

In [31]:
segmented_rfm = rfmTable

## The lowest recency, highest frequency and monetary amounts are our best customers.

In [32]:
def RScore(x,p,d):
    if x <= d[p][0.25]:
        return 1
    elif x <= d[p][0.50]:
        return 2
    elif x <= d[p][0.75]: 
        return 3
    else:
        return 4
    
def FMScore(x,p,d):
    if x <= d[p][0.25]:
        return 4
    elif x <= d[p][0.50]:
        return 3
    elif x <= d[p][0.75]: 
        return 2
    else:
        return 1

## Add segment numbers to the newly created segmented RFM table

In [33]:
segmented_rfm['r_quartile'] = segmented_rfm['recency'].apply(RScore, args=('recency',quantiles,))
segmented_rfm['f_quartile'] = segmented_rfm['frequency'].apply(FMScore, args=('frequency',quantiles,))
segmented_rfm['m_quartile'] = segmented_rfm['monetary_value'].apply(FMScore, args=('monetary_value',quantiles,))
segmented_rfm.head()

,rContactID,recency,frequency,monetary_value,r_quartile,f_quartile,m_quartile
0,23,388,1,2.500000e+02,4,4,4
1,24,481,3,6.522625e+02,4,1,4
2,25,13,14,5.245400e+04,1,1,1
3,26,64,1,1.504500e+06,2,4,1
4,27,4,2,2.279300e+04,1,2,2


## Add a new column to combine RFM score: 111 is the highest score as we determined earlier.

In [34]:
segmented_rfm['RFMScore'] = segmented_rfm.r_quartile.map(str) + segmented_rfm.f_quartile.map(str) + segmented_rfm.m_quartile.map(str)
segmented_rfm.head()

,rContactID,recency,frequency,monetary_value,r_quartile,f_quartile,m_quartile,RFMScore
0,23,388,1,2.500000e+02,4,4,4,444
1,24,481,3,6.522625e+02,4,1,4,414
2,25,13,14,5.245400e+04,1,1,1,111
3,26,64,1,1.504500e+06,2,4,1,241
4,27,4,2,2.279300e+04,1,2,2,122


# -Top customers

In [35]:
segmented_rfm[segmented_rfm['RFMScore']=='111'].sort_values('monetary_value', ascending=False).head(10)

,rContactID,recency,frequency,monetary_value,r_quartile,f_quartile,m_quartile,RFMScore
2620,14694,7,12,5.288769e+06,1,1,1,111
9780,76751,9,5,2.814533e+06,1,1,1,111
879,3456,2,5,2.205067e+06,1,1,1,111
61,122,2,4,2.187000e+06,1,1,1,111
12726,105802,10,3,2.166250e+06,1,1,1,111
5412,37723,2,5,1.523378e+06,1,1,1,111
210,462,3,10,1.465142e+06,1,1,1,111
3522,21317,22,8,1.423667e+06,1,1,1,111
2282,12361,2,9,1.367212e+06,1,1,1,111
15658,138555,2,4,1.344000e+06,1,1,1,111


In [39]:
rfmTable.sample(30)

,rContactID,recency,frequency,monetary_value,r_quartile,f_quartile,m_quartile,RFMScore
8313,63716,86,1,150.500000,2,4,4,244
16210,146492,10,0,500.000000,1,4,4,144
2905,16663,34,0,15000.000000,2,4,2,242
6164,44887,284,1,1000.000000,4,4,3,443
15733,139778,83,1,250.000000,2,4,4,244
9289,72427,254,1,7400.000000,4,4,2,442
1129,4903,454,1,500.000000,4,4,4,444
7507,56817,131,2,800.000000,3,2,4,324
13098,109543,181,6,15115.571429,3,1,2,312
16759,152837,2,1,70000.000000,1,4,1,141


In [38]:
#rfm Analysis

# m good (1,2) and f bad (4,3)

In [63]:
 m12_f43=rfmTable.query('(m_quartile == "1" or m_quartile == "2") and (f_quartile == "4" or f_quartile == "3")')

In [74]:
m12_f43 = m12_f43.merge(sav[['rContactID','saving_duration']], on= 'rContactID', how='inner')

In [87]:
savings[savings['rContactID']==46]

,rContactID,Account_Number,Balance,Transaction_Type,Total_Payment,Transaction_ID,Status,saving_cycle,transaction_date,post_balance,prev_balance,is_less_200Ar_and_withdrawed,cycle,nth_cycle,mean_post_balance_per_cycle,last_payment
1299,46,00000000023,20000.0,WalletToAccount,20000.0,152307184196319735,Success,1,2018-04-07 06:31:39.420,20000.0,0.0,0,0,0,20000.0,2018-04-07 06:31:39.420


In [86]:
m12_f43[['rContactID','saving_duration','monetary_value','RFMScore']]

,rContactID,saving_duration,monetary_value,RFMScore
0,26,432,1504500.0,241
1,31,9,100000.0,441
2,34,114,6000.0,442
3,40,288,182700.0,341
4,42,82,8250.0,442
5,46,0,20000.0,442
6,47,99,350000.0,441
7,59,6,30000.0,441
8,65,24,5050.0,442
9,66,0,10000.0,442


In [80]:
m12_f43[['recency','frequency','monetary_value','saving_duration']].describe()

,recency,frequency,monetary_value
count,4329.000000,4329.000000,4.329000e+03
mean,156.022869,0.589282,1.024003e+05
std,148.962063,0.492021,3.611696e+05
min,2.000000,0.000000,5.000500e+03
25%,18.000000,0.000000,1.123500e+04
50%,107.000000,1.000000,2.930000e+04
75%,278.000000,1.000000,7.710000e+04
max,510.000000,1.000000,9.295200e+06


In [64]:
m12_f43.describe()

,rContactID,recency,frequency,monetary_value,r_quartile,f_quartile,m_quartile
count,4329.000000,4329.000000,4329.000000,4.329000e+03,4329.000000,4329.0,4329.000000
mean,72339.122430,156.022869,0.589282,1.024003e+05,2.477477,4.0,1.482559
std,51064.969101,148.962063,0.492021,3.611696e+05,1.203435,0.0,0.499753
min,26.000000,2.000000,0.000000,5.000500e+03,1.000000,4.0,1.000000
25%,26437.000000,18.000000,0.000000,1.123500e+04,1.000000,4.0,1.000000
50%,65428.000000,107.000000,1.000000,2.930000e+04,2.000000,4.0,1.000000
75%,115157.000000,278.000000,1.000000,7.710000e+04,4.000000,4.0,2.000000
max,171585.000000,510.000000,1.000000,9.295200e+06,4.000000,4.0,2.000000


In [40]:
rfmTable[(rfmTable['monetary_value']==1) &]

,rContactID,recency,frequency,monetary_value,r_quartile,f_quartile,m_quartile,RFMScore
512,1628,458,1,1.0,4,4,4,444
1030,4361,494,0,1.0,4,4,4,444
1166,5235,235,1,1.0,3,4,4,344
1268,5783,487,1,1.0,4,4,4,444
4213,26983,368,2,1.0,4,2,4,424
4946,33666,222,1,1.0,3,4,4,344


In [37]:
savings.rContactID.nunique()

18067

# ----------------------------------------------------------------------------------------
# ABC Testing

In [ ]:
Atest = segmented_rfm[(segmented_rfm['f_quartile']== 4 ) | (segmented_rfm['f_quartile']== 3)] 
Atest.rContactID.nunique() #

In [ ]:
Btest = rfmTable[(segmented_rfm['f_quartile']== 2 ) | (segmented_rfm['f_quartile']== 1)]
Btest.rContactID.nunique()

In [ ]:
Ctest = rfmTable[segmented_rfm['r_quartile']== 4]
Ctest

In [ ]:
Atest= Atest.append(Btest)

In [ ]:
Atest = Atest.append(Ctest)

In [ ]:
Atest.to_excel('ABC_testing.xlsx')

In [ ]:
savings[['rContactID','saving_duration','cycle','post_balance','max_post_balance_per_cycle']][savings['rContactID']==26] #.Total_Payment.sum() #

In [ ]:
f_score= []
m_score= []
r_score= []

columns = ['frequency', 'monetary']
scores_str =['f_score', 'm_score']
scores = [f_score, m_score]

for n in range(len(columns)):
    # Order by columns: #those with high monetary values and high frequency will have higher score, that is good.
    rfmTable = rfmTable.sort_values(columns, ascending=False)
    
    # Create new index
    print("creating refs")
    refs = np.arange(1,savings.rContactID.nunique()+1)
    rfmTable['refs'] = refs
    step = np.floor(savings.rContactID.nunique()/5) 
    
        # Add score
    for i, row in rfmTable.iterrows(): # this syntax is because we did not reset index after groupby
        if row['refs'] <= step:
            scores[n].append(5)
        elif row['refs'] > step and row['refs'] <= step*2:
            scores[n].append(4)
        elif row['refs'] > step*2 and row['refs'] <= step*3:
            scores[n].append(3)
        elif row['refs'] > step*3 and row['refs'] <= step*4:
            scores[n].append(2)
        else: 
            scores[n].append(1)
            
    # Create f_score column
    rfmTable[scores_str[n]] = scores[n]
    rfmTable.head(2)
    
# For recency, we do the opposite: most recents are better, so we order as ascending
rfmTable = rfmTable.sort_values('recency', ascending=True)
    
# Recreate index
refs = np.arange(1,savings.rContactID.nunique()+1)
rfmTable['refs'] = refs 
    
# Add score
for i, row in rfmTable.iterrows():
    if row['refs'] <= step:
        r_score.append(5)
    elif row['refs'] > step and row['refs'] <= step*2:
        r_score.append(4)
    elif row['refs'] > step*2 and row['refs'] <= step*3:
        r_score.append(3)
    elif row['refs'] > step*3 and row['refs'] <= step*4:
        r_score.append(2)
    else: 
        r_score.append(1)

# Create r_score column
rfmTable['r_score'] = r_score
    
rfmTableScores = rfmTable.drop(['frequency', 'monetary', 'recency', 'refs'], axis=1)

rfmTableScores.head(5)

In [ ]:
rfmTableScores[(rfmTableScores['m_score']==3) & (rfmTableScores['f_score']==5)]

In [ ]:
savings[savings['rContactID']==114097].post_balance.max()

In [ ]:
savings[savings['rContactID']==150600].post_balance.max()

In [ ]:
rfmTable

In [ ]:
rfmTableScores.describe()

# -------------------------------------------------------------------------------------------------------------------

In [ ]:
plt.style.use('seaborn-whitegrid')

sav.hist(bins=10, figsize=(14,10), color='#E14906')
plt.show()

In [ ]:
plt.figure(figsize=(12,8))
sns.scatterplot(sav['saving_duration'],y=(sav['balance']) )

#### There are some worth tracking number of people with around 2,000,000 Ar savings. Need to explore them

# - Study of the less than 200 Ar balance

##### 12882 customers in this segment

#### We can see that 75% of our customers have less 0 Ar in their balance  withing 124 days while half of them have 0 Ar in their balance for 46 days. 
### 'This dataframe is useful for the customer segmentation for the Marketing campaign, and consider only the positive balance.'

In [ ]:
between0_200 =sav[(sav['balance']>=0)&(sav['balance']<=200)]

In [ ]:
between0_200.describe()

In [ ]:
sns.distplot(between0_200.balance)

In [ ]:
sns.distplot(between0_200.cycle_max)

In [ ]:
plt.hist(between0_200.saving_duration,bins = 20)

### the 200 Ar customers will be asked to save 1000 Ar for a certain period, will give us about 12,862,000 Ar

# A- Campagne de sensibilisation

# A-1) between 200Ar and 2000 Ar

##### Il y a 2093 clients  (75% of customeres have 1000 Ar, 1 cycle, and save within 82 days)

In [ ]:
between200_2000 = sav[(sav['balance']>200) & (sav['balance']<=2000)]

In [ ]:
between200_2000.describe()

In [ ]:
2094*1000

# A-2) between 2000 and 5000 Ar

552 customers (75% have 5000 Ar, 159 days saving, and 1 cycle)

In [ ]:
between2000_5000 = sav[(sav['balance']>2000) & (sav['balance']<=5000)]

between2000_5000.describe()

# B- Campagne bonification du top5

# B-1) between 5000 and 10000 Ar

393 customers (75% have 10000 Ar, 1 cycle, and 154 saving days)

In [ ]:
between5000_10000 = sav[(sav['balance']>5000) & (sav['balance']<=10000)]

between5000_10000.describe()

# B-2) between 10000 to 20000
256 customers and 75% of them have 19625 Ar, and 2 cycle, 239 saving days

In [ ]:
between10k_20k = sav[(sav['balance']>10000) & (sav['balance']<=20000)]
between10k_20k.describe()

# B-3) between 20000 and 40000
192 customers, 50% have 30000 Ar, 99 days of saving duration

In [ ]:
between20k_40k = sav[(sav['balance']>20000) & (sav['balance']<=40000)]
between20k_40k.describe()

# B-4) between 40k and 60k
107 customers  

In [ ]:
between40k_60k = sav[(sav['balance']>40000) & (sav['balance']<=60000)]
between40k_60k.describe()

# B-5) between 60k and 80k
48 customers

In [ ]:
between60k_80k = sav[(sav['balance']>60000) & (sav['balance']<=80000)]
between60k_80k.describe()

# B-6) between 80k and 100k
59 customers

In [ ]:
between80k_100k = sav[(sav['balance']>80000) & (sav['balance']<=100000)]
between80k_100k.describe()

# B-7) between 100k and 150k
62 customers

In [ ]:
between100k_150k = sav[(sav['balance']>100000) & (sav['balance']<=150000)]
between100k_150k.describe()

# B-8) between 150k and 200k
36 customers

In [ ]:
between150k_200k = sav[(sav['balance']>150000) & (sav['balance']<=200000)]
between150k_200k.describe()

# B-9) between 200k and 300k
47 customers

In [ ]:
between200k_300k = sav[(sav['balance']>200000) & (sav['balance']<=300000)]
between200k_300k.describe()

# B-10) between 300k and 500k
52 customers

In [ ]:
between300k_500k = sav[(sav['balance']>300000) & (sav['balance']<=500000)]
between300k_500k.describe()

In [ ]:
7.916245e+05

# C- Campagne speciale

# C-1) between 500k and 1M5
50 customers

In [ ]:
between500k_1M5 = sav[(sav['balance']>500000) & (sav['balance'] <=1500000)]

between500k_1M5.describe()

# C-2) between 1M5 and 4M
14 customers

In [ ]:
between1M5_4M = sav[(sav['balance']>1500000) & (sav['balance'] <=4000000)]

between1M5_4M.describe()

In [ ]:
# C-3)  4M et plus 14 customers

In [ ]:
between4M_10M = sav[(sav['balance']>4000000) & (sav['balance'] <=10000000)]

between4M_10M.describe()

In [ ]:
x = sav[sav['balance']>20000]

In [ ]:
x.describe()

In [ ]:
2.010000e+04

### (TSY ILAINA INTSONY) 2-2) A new table for the rContactID behavior study

In [ ]:
study = savings.groupby(['rContactID','Transaction_Date']).agg(
    {'Total_Payment':('min','max','sum'), 'prev_balance':'sum', 'post_balance':'sum'
    }) #.cumsum() #agg({'Total_Payment':'cumsum'})
study.columns = ["_".join(x) for x in study.columns.ravel()]
#dd.columns = dd.columns.droplevel(0)
study = study.reset_index()
study.rename(columns={'Total_Payment_sum':'balance'}, inplace = True)

In [ ]:
study

In [ ]:
#study['transaction_frequency']=
#not useful anymore
#study.groupby('rContactID')['Total_Payment_count'].sum()

In [ ]:
study = study.merge(sav[['rContactID','saving_duration', 'Transaction_Date_min', 'Transaction_Date_max']], how = 'inner',  on='rContactID')

In [ ]:
savings.dtypes

In [ ]:
study = study.merge(savings[['rContactID','Transaction_Time','cycle','nth_cycle']])

In [ ]:
savings[savings['rContactID']==24].head(5)

In [ ]:
study[study['rContactID']==24].head(4)

In [ ]:
columnsTitles=[ "rContactID","Transaction_Date","Transaction_Time","Total_Payment_count","Total_Payment_min", "Total_Payment_max", "prev_balance_sum", "post_balance_sum","balance", "Transaction_Date_min", "Transaction_Date_max","saving_duration"]
study=study.reindex(columns=columnsTitles)

In [ ]:
study = study.sort_values(["Transaction_Time"])

In [ ]:
study

### Misy tsy milamina eto

In [ ]:
savings[savings['rContactID']==25]

In [ ]:
transaction_activity

### The median of the saving duration is 41 days 

# -------------------------------------------------------------------------------------------------------------------

# Saving Campaign

In [ ]:
campaign = savings.groupby('rContactID').agg(
    {'Total_Payment':'sum','Transaction_Date':('min', 'max')})
campaign.columns = ["_".join(x) for x in campaign.columns.ravel()]
#dd.columns = dd.columns.droplevel(0)
campaign = campaign.reset_index()
campaign.rename(columns={'Total_Payment_sum':'balance'}, inplace = True)

#campaign['saving_duration'] = campaign['Transaction_Date_max']-campaign['Transaction_Date_min'] 

#campaign.saving_duration = campaign.saving_duration.apply(lambda x: str(x))

#campaign.saving_duration = campaign.saving_duration.apply(lambda x: str.split(x)[0])

#campaign.saving_duration = campaign.saving_duration.apply(lambda x: int(x))

### Those with balance > 0

In [ ]:
campaign1 = campaign[campaign.balance>0]

len(campaign1)

In [ ]:
campaign1.describe()

### Those with balance > 200

In [ ]:
campaign2= campaign[campaign.balance>200]

len(campaign2)

In [ ]:
campaign2.describe()

In [ ]:
1.000000e+04

### Those with balance > 2000 

In [ ]:
campaign3= campaign[campaign.balance>=2000]

len(campaign3)

In [ ]:
campaign3.describe()

In [ ]:
4.497900e+04

### Buckets:
##### Q1: [2000,10000[ Ar -
##### Q2: [10000, 40000 [ Ar - 
##### Q3: [40000,et plus[ Ar -

#### -> Those are the active customers nowadays. There are 3901 customers.

#### -> 50% of them have 2000 Ar in their balance within a period of a 25 days,  while 75 % of them have 10000 Ar in their balance for 5 months

In [ ]:
sns_plot = sns.distplot(sav.balance)
sns_plot.figure.savefig("distribution_saving.png")

In [ ]:
plt.figure(figsize=(13,8))
sns.distplot(campaign.balance)
plt.hist(campaign.balance, color = 'blue', edgecolor = 'black', bins = 200)
plt.savefig('customer_distribution.png')

# ------------------------------------------------------------------------------------------------------------------

In [ ]:
"""
dd_cycle = study[study].groupby('rContactID').agg(
    {"Transaction_Date":"max"
        
    }
    )
dd_cycle.rename({'Transaction_Date':'last_payment'}, axis=1, inplace=True)

dd_cycle = dd_cycle.reset_index()
"""

In [ ]:
"""
withdraw = study[study['post_balance_sum']<=0]# .groupby('rContactID').count()

#cycle['payment_date_for_cycle']=
deposit = study[study['post_balance_sum']>0]

withdraw

deposit
"""

In [ ]:
study.describe()

In [ ]:
custom_list = [study['rContactID'][i] for i in range(len(study))]

In [ ]:
custom_list = list(set(custom_list))

In [ ]:
date_withdraw = []
date_deposit = []

In [ ]:
savings[savings['rContactID'] == 25].head()

In [ ]:
study[study['rContactID']==25].head(2)

In [ ]:
 study['rContactID']['post_balance_sum']<=0

In [ ]:
temp = study[study['rContactID']==25 ]

In [ ]:
temp[temp.post_balance_sum<0]

In [ ]:
temp

In [ ]:
x=study[(study['rContactID']== 98305) & (study['post_balance_sum']<=0)]

In [ ]:
len(x)

In [ ]:
x=x.reset_index()
x[x['post_balance_sum']<=0]

In [ ]:
x.Transaction_Date.dt.date

In [ ]:
withdraw = {}
for c in custom_list:
    if c not in withdraw.keys():      
        temp = study[(study['rContactID'] == c) & (study['post_balance_sum']<=0)]
        temp.reset_index(inplace=True)
        for i in range(len(temp)):
            time_withdraw.append(temp.Transaction_Date[i])
        withdraw[c] = time_withdraw 
        time_withdraw =[]

In [ ]:
withdraw[98305]

In [ ]:
deposit = {}
for c in custom_list:
    if c in deposit.keys()
        temp = study[study['rContactID'] == c]
        temp.reset_index(inplace=True)
        temp[(temp.post_balance_sum > 0)]
        time_deposit.append(temp.Transaction_Date[0])
        deposit[c] = time_deposit 
    

In [ ]:
study[study['rContactID']==98305]

In [ ]:
withdraw_df = pd.DataFrame.from_dict(withdraw)


In [ ]:
withdraw_df

In [ ]:
study.head(10)

In [ ]:
all_last_payment_done = study[(study['last_payment'] == study['Transaction_Date'])]

In [ ]:
all_last_payment_done

## Customers leaving:
### Keys:
   ### customers leave when balance is around 200, and the last payment is a withdrawal (i.e last payment <0)


In [ ]:
quit_customers=study[(study['post_balance_sum']<=200) & (all_last_payment_done.Total_Payment_min < 0)]

In [ ]:
len(quit_customers)

#### 13370 customers have saved for some time and left

In [ ]:
len(savings)

In [ ]:
quit_customers.rContactID

In [ ]:
#savings.groupby('rContactID', as_index=False)[['Total_Payment']].cumsum() #.agg({'Total_Payment':'cumsum'})

In [ ]:
savings[savings['rContactID'] == 2088]

#### This particular customer (rContactID = 2088) have started saving from 2018-05-15 until 2019-07-05 (1 year and 3 months)

In [ ]:
### Does not work

fig = ff.create_facet_grid(
    savings,
    x='duration',
    y='balance',
    color_name='marital',
    show_boxes=False,
    marker={'size': 10, 'opacity': 1.0},
    colormap={'single': 'rgb(165, 242, 242)', 'married': 'rgb(253, 174, 216)', 'divorced': 'rgba(201, 109, 59, 0.82)'}
)

iplot(fig, filename='facet - custom colormap')

In [ ]:

#dd = savings[savings["Total_Payment"]>0]
sav = savings.groupby('rContactID').agg(
    {'Total_Payment':'sum','Transaction_Date':('min', 'max')})
sav.columns = ["_".join(x) for x in sav.columns.ravel()]
#dd.columns = dd.columns.droplevel(0)
sav = sav.reset_index()
sav.rename(columns={'Total_Payment_sum':'balance'}, inplace = True)

In [ ]:
sav.describe()

#### 75% of the customers have at most 200 Ar 

In [ ]:
sav[sav['balance']<-2000].head(2)

## Median

In [ ]:
sav.balance.median()

#### The median shows that half of the customers do not even have anything in their balance

In [ ]:
len(sav[(sav['balance']>=0) & (sav['balance']<=200)])

#### 12684 customers have at most 200 Ar

2536800 Ar saved by the 200Ar customers

### Length of saving

In [ ]:
sav['saving_duration'] = sav['Transaction_Date_max']-sav['Transaction_Date_min'] 

sav.saving_duration = sav.saving_duration.apply(lambda x: str(x))

sav.saving_duration = sav.saving_duration.apply(lambda x: str.split(x)[0])

sav.saving_duration = sav.saving_duration.apply(lambda x: int(x))

In [ ]:
sav[sav["rContactID"] == 2088]

In [ ]:
sav.describe()

#### The median shows that half of the customers save until 41 days.
#### 75% have 200 Ar for 127 days (at most) (~ 4 months )

### Interpretation

##### Those 12684 customers seem to quit saving, and some of them even leave their account inactive within 4 months.

##### (We will see when do they come back and how many do come back start using the saving service again.)

### But let's see first those 12684 customers

In [ ]:
low_sav = sav[(sav['balance']>=0) & (sav['balance']<=200)]

In [ ]:
low_sav.head()

In [ ]:
low_sav.dtypes

In [ ]:
plt.figure(figsize=(12,8))
sns.scatterplot(low_sav['saving_duration'],y=( low_sav['balance']) )

In [ ]:
low_sav.dtypes

In [ ]:
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')

low_sav.hist(bins=10, figsize=(14,10), color='#E14906')
plt.show()

## 2- When do customers come back to saving

In [ ]:
savings.head(3)

In [ ]:
savings[savings['rContactID']==3622]

In [ ]:
savings[savings['rContactID']==10301]

In [ ]:
savings[savings['rContactID']==10301]['post_balance'].hist()

In [ ]:
savings[savings['rContactID']==2088]['Total_Payment'].hist()

In [ ]:
savings.head(10)

In [ ]:
study=savings.groupby(['rContactID','Transaction_Date']).agg(
    {'Total_Payment':'sum','Transaction_Date':('min', 'max')})
study.columns = ["_".join(x) for x in study.columns.ravel()]
#dd.columns = dd.columns.droplevel(0)
study = study.reset_index()
study.rename(columns={'Total_Payment_sum':'balance'}, inplace = True)

In [ ]:
savings

In [ ]:
def 

In [ ]:
savings['new_transaction_type'] = savings['Transaction_Type'].apply(#depot or withdraw)

## 3- Etude du Depot

In [ ]:
savings[savings["Total_Payment"]>0].head(6)

In [ ]:
savings.describe()

## Let's consider only those who have balance >0

In [ ]:
dd = savings[savings["Total_Payment"]>0]

In [ ]:
dd.head(2)

In [ ]:
import numpy as np
#dd = savings[savings["Total_Payment"]>0]
dd = dd.groupby('rContactID').agg(
    {'Total_Payment':'sum','Transaction_Date':('min', 'max')})
dd.columns = ["_".join(x) for x in dd.columns.ravel()]
#dd.columns = dd.columns.droplevel(0)
dd = dd.reset_index()

In [ ]:
dd.rename(columns={'Total_Payment_sum':'balance'}, inplace = True)

In [ ]:
dd.describe()

#### the mean is misleading 

## Median of the balance

In [ ]:
dd.balance.median()

In [ ]:
dd.dtypes

## mean of payment for balance > 0 is 228687.3 Ar and 50% of them have 10900.0 while 75% have 80000.0

In [ ]:
len(dd[dd['balance']>=10000])

### 8843 customers have more than 10000 Ar in their balance.  
### So we can make at least 88430000 Ar from them if we ask them to save 10000 Ar.

# Length of saving duration

In [ ]:
dd['saving_duration'] = dd['Transaction_Date_max']-dd['Transaction_Date_min'] 

In [ ]:
dd.describe()

In [ ]:
dd.saving_duration.median()

#### We can see that people make a saving of 10900 Ar in 31 days, so in one month, we can make 88430000 Ar for saving of 10000 Ar per month per person

## joining the rContactID with their behaviors

In [ ]:
dd = dd.merge(savings, how='left', left_on='rContactID' ,right_on='rContactID')

In [ ]:
dd = dd.drop(columns=['Transaction_Date','Transaction_Time','Transaction_ID','Account_Number', 'Balance', 'Status', 'Total_Payment'], axis = 1)

In [ ]:
dd